In [73]:
import pandas as pd
import geopandas as gpd
import requests
from datetime import date
import plotly.express as px
import numpy as np
import os
from gjf.geojson_fixer import validity
from gjf.geojson_fixer import apply_fixes_if_needed
import json


In [2]:
# this is importing a local config file with api keys.
# if you do not have access to the author's api keys, you should obtain your own and replace in your environment
import config
ipc_api_key = "&key="+config.ipc_key

1. get all analysis ids
2. make call to each analysis id and copy the geo data into data frame "areas/12856213/P?format=geojson"
3. do this for for each type of single letter analysis type if we want to do chronic and acute. probably just do acute for now
4. figure out the difference between period codes?
   
there are two types of analysis types, A and C.
there are three types of single letter period codes C, P, and 

relevant URLs:


Returns geojson https://api.ipcinfo.org/areas/12856213/P?format=geojson&key=9fa11565-2a55-4c5b-b40a-bf2fe5bda1fb

Gets analysis list https://api.ipcinfo.org/analyses?country=AF&year=2020&type=A&key=9fa11565-2a55-4c5b-b40a-bf2fe5bda1fb

In [6]:
url_base = "https://api.ipcinfo.org/"

def get_ipc_data(endpoint, format):
    request = requests.get(url=url_base+endpoint+ipc_api_key)
    # print(request)
    #  print ("length is " + str(len(request.json()["dataset"])))
    if format == "pd":
        return pd.DataFrame(request.json())
    elif format == "json":
        return request.json()
    elif format == "gpd":
        return gpd.GeoDataFrame(request.json())
    else:
        return request.text

In [7]:
all_analysis = get_ipc_data("analyses?format=json&type=A", "pd")
# make id the index
all_analysis.set_index("id", inplace=True)
all_analysis

<Response [200]>


,title,link,country,year,condition,created,modified
id,,,,,,,
12135258,Burundi - Acute Food Insecurity March 2017 (20...,http://www.ipcinfo.org/ipc-country-analysis/de...,BI,2017,A,2017-11-20,2022-02-15
12135625,Central African Republic - Acute Food Insecuri...,http://www.ipcinfo.org/ipc-country-analysis/de...,CF,2017,A,2017-11-20,2022-08-17
12138309,Guatemala - Acute Food Insecurity July 2017 (D...,http://www.ipcinfo.org/ipc-country-analysis/de...,GT,2017,A,2017-11-20,2022-02-15
12140917,Haiti - Acute Food Insecurity February 2017,http://www.ipcinfo.org/ipc-country-analysis/de...,HT,2017,A,2017-11-20,2022-02-15
12144729,Eswatini - Acute Food Insecurity July 2017,http://www.ipcinfo.org/ipc-country-analysis/de...,SZ,2017,A,2017-11-20,2022-02-15
...,...,...,...,...,...,...,...
26138091,Madagascar - Acute Food Insecurity July 2022 (...,https://www.ipcinfo.org/ipc-country-analysis/d...,MG,2022,A,2022-08-16,2022-09-16
26178025,Burundi - Acute Food Insecurity August 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,BI,2022,A,2022-08-31,2022-10-04
26190751,Somalia - Acute Food Insecurity August 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,SO,2022,A,2022-09-11,2022-10-05


So P endpoint is returning invalid geojson with some variables not having quotes around them and some having single quotes when they should have double. C endpoing seems valid?

In [15]:
# testing the C endpiont for all IDs
passed = []
failed = []
count = 1
for id in all_analysis.index:
    # print country column and condition column that matches id
    print ("processing entry " + str(count))
    file_name = str(id) + "_" + str(all_analysis.loc[id, "year"]) + "_" + all_analysis.loc[id, "country"] + "_C-ipc_period"
    try:
        analysis_json = get_ipc_data(f"areas/{id}/C?format=geojson", "json")
        
        analysis_gdf = gpd.GeoDataFrame.from_features(analysis_json)
        # save to file
        print (id)
        analysis_gdf.to_file(f"../assets/ipc-data/{file_name}.geojson", driver="GeoJSON")
        passed.append(id)
        print (file_name, " passed")
    except:
        failed.append(id)
        print (file_name, " failed")
    count += 1
print ("passed: " + str(len(passed)))
print ("failed: " + str(len(failed)))

processing entry 1
<Response [200]>
12135258
12135258_2017_BI_C-ipc_period  passed
processing entry 2
<Response [200]>
12135625
12135625_2017_CF_C-ipc_period  passed
processing entry 3
<Response [200]>
12138309
12138309_2017_GT_C-ipc_period  passed
processing entry 4
<Response [200]>
12140917
12140917_2017_HT_C-ipc_period  passed
processing entry 5
<Response [200]>
12144729
12144729_2017_SZ_C-ipc_period  passed
processing entry 6
<Response [200]>
12148833
12148833_2017_HT_C-ipc_period  passed
processing entry 7
<Response [200]>
12151682
12151682_2017_MG_C-ipc_period  passed
processing entry 8
<Response [200]>
12151843
12151843_2017_MG_C-ipc_period  passed
processing entry 9
<Response [200]>
12152173
12152173_2017_LS_C-ipc_period  passed
processing entry 10
<Response [200]>
12152462
12152462_2017_MW_C-ipc_period  passed
processing entry 11
<Response [200]>
12166797
12166797_2017_AF_C-ipc_period  passed
processing entry 12
<Response [200]>
12166890
12166890_2017_AF_C-ipc_period  passed
p

In [23]:
# number of items in passed
print(len(passed))
print(len(failed))

178
20


In [33]:
# create a list of all files in ../assets/ipc-data
files = os.listdir("../assets/ipc-data")
files

['25159242_2022_UG_C-ipc_period.geojson',
 '26468816_2022_HT_C-ipc_period.geojson',
 '24755659_2022_SO_C-ipc_period.geojson',
 '12956017_2018_BI_C-ipc_period.geojson',
 '26094569_2022_LS_C-ipc_period.geojson',
 '12148833_2017_HT_C-ipc_period.geojson',
 '26096991_2022_ZM_C-ipc_period.geojson',
 '13853542_2019_MG_C-ipc_period.geojson',
 '18560673_2020_CD_C-ipc_period.geojson',
 '14010107_2019_CD_C-ipc_period.geojson',
 '12859540_2018_LS_C-ipc_period.geojson',
 '19526876_2020_YE_C-ipc_period.geojson',
 '24259254_2021_AO_C-ipc_period.geojson',
 '18584762_2020_NA_C-ipc_period.geojson',
 '16898810_2020_MZ_C-ipc_period.geojson',
 '14116618_2019_HT_C-ipc_period.geojson',
 '24025678_2021_BI_C-ipc_period.geojson',
 '12934754_2018_YE_C-ipc_period.geojson',
 '13928767_2019_AF_C-ipc_period.geojson',
 '13602488_2019_ZW_C-ipc_period.geojson',
 '12908433_2018_MZ_C-ipc_period.geojson',
 '24004104_2021_SD_C-ipc_period.geojson',
 '18543757_2020_MW_C-ipc_period.geojson',
 '13052452_2019_SO_C-ipc_period.ge

In [50]:

# create a geo data frame with each of the files in files
joined_gdf = gpd.GeoDataFrame()
for file in files:
    try:
        joined_gdf = joined_gdf.append(gpd.read_file("../assets/ipc-data/" + file))
    except:
        print (file)
joined_gdf

/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_18454/3731268607.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_gdf = joined_gdf.append(gpd.read_file("../assets/ipc-data/" + file))
/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_18454/3731268607.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_gdf = joined_gdf.append(gpd.read_file("../assets/ipc-data/" + file))
/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_18454/3731268607.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_gdf = joined_gdf.append(gpd.read_file("../assets/ipc-data/" + file))
/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_18454/3731268607.py:5: FutureWarning: The frame.append method 

.DS_Store


/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_18454/3731268607.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_gdf = joined_gdf.append(gpd.read_file("../assets/ipc-data/" + file))
/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_18454/3731268607.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_gdf = joined_gdf.append(gpd.read_file("../assets/ipc-data/" + file))
/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_18454/3731268607.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_gdf = joined_gdf.append(gpd.read_file("../assets/ipc-data/" + file))
/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_18454/3731268607.py:5: FutureWarning: The frame.append method 

ipc_v1.mbtiles


/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_18454/3731268607.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_gdf = joined_gdf.append(gpd.read_file("../assets/ipc-data/" + file))
/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_18454/3731268607.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_gdf = joined_gdf.append(gpd.read_file("../assets/ipc-data/" + file))
/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_18454/3731268607.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joined_gdf = joined_gdf.append(gpd.read_file("../assets/ipc-data/" + file))
/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_18454/3731268607.py:5: FutureWarning: The frame.append method 

,aar_id,confidence_level,ipc_period,overall_phase,population_min,population_percentage_min,justification,prolonged_crisis,population_phase_worse,population_percentage_phase_worse,...,phase3_population,phase3_percent,phase3_color,phase4_population,phase4_percent,phase4_color,phase5_population,phase5_percent,phase5_color,geometry
0,25159255,3,C,3,60410,0.35,Abim district is classified in IPC phase 3 wi...,False,None,None,...,51780.0,0.30,#E67800,8630.0,0.05,#C80000,0.0,0.0,#640000,"POLYGON ((33.61183 3.13407, 33.61697 3.13486, ..."
1,25159261,3,C,3,43830,0.3,"In summary, its clearly evident that Amudat Di...",False,None,None,...,36525.0,0.25,#E67800,7305.0,0.05,#C80000,0.0,0.0,#640000,"POLYGON ((34.92685 2.28660, 34.93040 2.28508, ..."
2,25159253,3,C,3,70600,0.5,"In Kaabong district, 50% of the population are...",False,None,None,...,56480.0,0.40,#E67800,14120.0,0.10,#C80000,0.0,0.0,#640000,"POLYGON ((34.55892 3.11066, 34.55317 3.10693, ..."
3,25159245,3,C,3,21630,0.35,Karenga district has been classified to be in ...,False,None,None,...,18540.0,0.30,#E67800,3090.0,0.05,#C80000,0.0,0.0,#640000,"POLYGON ((33.96628 3.94338, 33.96833 3.93530, ..."
4,25159257,3,C,3,107650,0.5,"Kotido District has been rated at phase 3, whe...",False,None,None,...,96885.0,0.45,#E67800,10765.0,0.05,#C80000,0.0,0.0,#640000,"POLYGON ((33.72277 3.44022, 33.73358 3.43953, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,17950199,0,C,2,100177,0.1,"Au cours de la période actuelle, la zone n' a ...",False,None,None,...,100177.0,0.10,#E67800,0.0,0.00,#C80000,0.0,0.0,#640000,"MULTIPOLYGON (((30.10073 -2.42500, 30.09402 -2..."
4,17950205,0,C,2,221806,0.15,"Par rapport à 2019,les productions ont baissé ...",False,None,None,...,221806.0,0.15,#E67800,0.0,0.00,#C80000,0.0,0.0,#640000,"MULTIPOLYGON (((29.53632 -3.13268, 29.53458 -3..."
5,17950207,3,C,2,126162,0.15,La zone de l' Imbo est globalement classée en ...,False,None,None,...,84108.0,0.10,#E67800,42054.0,0.05,#C80000,0.0,0.0,#640000,"MULTIPOLYGON (((29.30253 -3.00879, 29.30293 -3..."
6,17950213,0,C,2,505966,0.15,Malgré une pluviométrie excessive au cours de ...,False,None,None,...,505966.0,0.15,#E67800,0.0,0.00,#C80000,0.0,0.0,#640000,"MULTIPOLYGON (((29.69014 -3.56946, 29.69503 -3..."


In [51]:
# print geometry type in joined_gdf
joined_gdf.geom_type.unique()

array(['Polygon', 'MultiPolygon', 'Point', 'GeometryCollection'],
      dtype=object)

In [53]:
# write joined_gdf to shape file with geopandas
joined_gdf.to_file('"../assets/ipc-data/joined_gdfa.shp"', driver='ESRI Shapefile')

/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_18454/4099118925.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  joined_gdf.to_file('"../assets/ipc-data/joined_gdfa.shp"', driver='ESRI Shapefile')


DriverError: Failed to create directory "../assets/ipc-data/joined_gdfa.shp" for shapefile datastore.

In [55]:
# print crs of joined_gdf
joined_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [57]:
# set joined_gdf crs to epsg:4326
joined_gdf = joined_gdf.to_crs(epsg=4326)

In [60]:
joined_gdf.to_file("../assets/ipc-data/joined_gdfa.shp")

/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_18454/702875884.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  joined_gdf.to_file("../assets/ipc-data/joined_gdfa.shp")


RuntimeError: GDAL Error: Attempt to write non-polygon (POINT) geometry to POLYGON type shapefile. 
 
 Failed to write record: {'id': '0', 'type': 'Feature', 'properties': {'aar_id': '25114535', 'confidence_level': 3, 'ipc_period': 'C', 'overall_phase': 3, 'population_min': '325081', 'population_percentage_min': '0.5', 'justification': "FCS is indicative of phase 4+ with 55.6% of people in phase 4 above and 38% in phase 3. rCSI also shows an IPC phase 4 situation with 42.8% of people in phase 3 and above, and 56.4% of people in IPC phase 2. HDDS shows IPC phase 3 situation with 18.8% in phase 4 and 49.2% in IPC phase 3. HHS is indicative of phase 4 with 22% of people in this phase and 49.2% in phase 3. Additionally, LCS shows phase 4 situation with 31.6% in phase 4 and 30.4% in phase 3. The PLSA data shows that 64% of households experienced at least some type of shock in the last 6 months in urban Balkh: Loss of employment: 58%; reduced income: 14%; Severe sickness or natural death of 

In [61]:
joined_gdf.to_file("../assets/ipc-data/joined_gdf.geojson", driver='GeoJSON')  

In [66]:
# read in ../assets/ipc-data/joined_gdf.geojson as json
with open("../assets/ipc-data/all_ipc_reports.geojson") as f:
    joined_gj = json.load(f)

In [75]:
fixed_gj = apply_fixes_if_needed(joined_gj)

In [76]:
validity(joined_gj)

('invalid',
 [['Valid Geometry',
   'Polygons and MultiPolygons should follow the right-hand rule'],
  ['Valid Geometry',
   'Polygons and MultiPolygons should follow the right-hand rule'],
  ['Valid Geometry',
   'Polygons and MultiPolygons should follow the right-hand rule'],
  ['Valid Geometry',
   'Polygons and MultiPolygons should follow the right-hand rule'],
  ['Valid Geometry',
   'Polygons and MultiPolygons should follow the right-hand rule'],
  ['Valid Geometry',
   'Polygons and MultiPolygons should follow the right-hand rule'],
  ['Valid Geometry',
   'Polygons and MultiPolygons should follow the right-hand rule'],
  ['Valid Geometry',
   'Polygons and MultiPolygons should follow the right-hand rule'],
  ['Valid Geometry',
   'Polygons and MultiPolygons should follow the right-hand rule'],
  ['Valid Geometry',
   'Polygons and MultiPolygons should follow the right-hand rule'],
  ['Valid Geometry',
   'Polygons and MultiPolygons should follow the right-hand rule'],
  ['Valid

In [94]:
# print name of each file in ../assets/ipc-data
v_list = []
for file in files:
    path = "../assets/ipc-data/" + file
    print (path)
    try:
        with open(path) as f:
            loaded_file = json.load(f)
            val = validity(loaded_file)
            v_list.append([file, val])
    except:
        print(file + " failed")

../assets/ipc-data/25159242_2022_UG_C-ipc_period.geojson
../assets/ipc-data/26468816_2022_HT_C-ipc_period.geojson
../assets/ipc-data/24755659_2022_SO_C-ipc_period.geojson
../assets/ipc-data/12956017_2018_BI_C-ipc_period.geojson
../assets/ipc-data/26094569_2022_LS_C-ipc_period.geojson
../assets/ipc-data/12148833_2017_HT_C-ipc_period.geojson
../assets/ipc-data/26096991_2022_ZM_C-ipc_period.geojson
../assets/ipc-data/13853542_2019_MG_C-ipc_period.geojson
../assets/ipc-data/18560673_2020_CD_C-ipc_period.geojson
../assets/ipc-data/14010107_2019_CD_C-ipc_period.geojson
../assets/ipc-data/12859540_2018_LS_C-ipc_period.geojson
../assets/ipc-data/19526876_2020_YE_C-ipc_period.geojson
../assets/ipc-data/24259254_2021_AO_C-ipc_period.geojson
../assets/ipc-data/18584762_2020_NA_C-ipc_period.geojson
../assets/ipc-data/16898810_2020_MZ_C-ipc_period.geojson
../assets/ipc-data/14116618_2019_HT_C-ipc_period.geojson
../assets/ipc-data/24025678_2021_BI_C-ipc_period.geojson
../assets/ipc-data/12934754_201

In [95]:
v_list

[['25159242_2022_UG_C-ipc_period.geojson',
  ('invalid',
   [['Valid Geometry',
     'Polygons and MultiPolygons should follow the right-hand rule'],
    ['Valid Geometry',
     'Polygons and MultiPolygons should follow the right-hand rule'],
    ['Valid Geometry',
     'Polygons and MultiPolygons should follow the right-hand rule'],
    ['Valid Geometry',
     'Polygons and MultiPolygons should follow the right-hand rule'],
    ['Valid Geometry',
     'Polygons and MultiPolygons should follow the right-hand rule'],
    ['Valid Geometry',
     'Polygons and MultiPolygons should follow the right-hand rule'],
    ['Valid Geometry',
     'Polygons and MultiPolygons should follow the right-hand rule'],
    ['Valid Geometry',
     'Polygons and MultiPolygons should follow the right-hand rule'],
    ['Valid Geometry',
     'Polygons and MultiPolygons should follow the right-hand rule']])],
 ['26468816_2022_HT_C-ipc_period.geojson',
  ('invalid',
   [['Valid Geometry',
     'Polygons and Mult

In [91]:
validity("assets/ipc-data/24755659_2022_SO_C-ipc_period.geojson")

TypeError: string indices must be integers

In [97]:

joined_edited = joined_gdf.drop(columns=["justification"])
joined_edited

,aar_id,confidence_level,ipc_period,overall_phase,population_min,population_percentage_min,prolonged_crisis,population_phase_worse,population_percentage_phase_worse,anl_id,...,phase3_population,phase3_percent,phase3_color,phase4_population,phase4_percent,phase4_color,phase5_population,phase5_percent,phase5_color,geometry
0,25159255,3,C,3,60410,0.35,False,None,None,25159242,...,51780.0,0.30,#E67800,8630.0,0.05,#C80000,0.0,0.0,#640000,"POLYGON ((33.61183 3.13407, 33.61697 3.13486, ..."
1,25159261,3,C,3,43830,0.3,False,None,None,25159242,...,36525.0,0.25,#E67800,7305.0,0.05,#C80000,0.0,0.0,#640000,"POLYGON ((34.92685 2.28660, 34.93040 2.28508, ..."
2,25159253,3,C,3,70600,0.5,False,None,None,25159242,...,56480.0,0.40,#E67800,14120.0,0.10,#C80000,0.0,0.0,#640000,"POLYGON ((34.55892 3.11066, 34.55317 3.10693, ..."
3,25159245,3,C,3,21630,0.35,False,None,None,25159242,...,18540.0,0.30,#E67800,3090.0,0.05,#C80000,0.0,0.0,#640000,"POLYGON ((33.96628 3.94338, 33.96833 3.93530, ..."
4,25159257,3,C,3,107650,0.5,False,None,None,25159242,...,96885.0,0.45,#E67800,10765.0,0.05,#C80000,0.0,0.0,#640000,"POLYGON ((33.72277 3.44022, 33.73358 3.43953, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,17950199,0,C,2,100177,0.1,False,None,None,17950196,...,100177.0,0.10,#E67800,0.0,0.00,#C80000,0.0,0.0,#640000,"MULTIPOLYGON (((30.10073 -2.42500, 30.09402 -2..."
4,17950205,0,C,2,221806,0.15,False,None,None,17950196,...,221806.0,0.15,#E67800,0.0,0.00,#C80000,0.0,0.0,#640000,"MULTIPOLYGON (((29.53632 -3.13268, 29.53458 -3..."
5,17950207,3,C,2,126162,0.15,False,None,None,17950196,...,84108.0,0.10,#E67800,42054.0,0.05,#C80000,0.0,0.0,#640000,"MULTIPOLYGON (((29.30253 -3.00879, 29.30293 -3..."
6,17950213,0,C,2,505966,0.15,False,None,None,17950196,...,505966.0,0.15,#E67800,0.0,0.00,#C80000,0.0,0.0,#640000,"MULTIPOLYGON (((29.69014 -3.56946, 29.69503 -3..."


In [98]:
# save joined edited to geojson
joined_edited.to_file("../assets/ipc-data/joined_no_justification.geojson", driver="GeoJSON")

In [99]:
# save joined edited to csv
joined_gdf.to_csv("../assets/ipc-data/all_ipc_reports.csv")


In [100]:
# list columns fo joined
joined_gdf.columns

Index(['aar_id', 'confidence_level', 'ipc_period', 'overall_phase',
       'population_min', 'population_percentage_min', 'justification',
       'prolonged_crisis', 'population_phase_worse',
       'population_percentage_phase_worse', 'anl_id', 'admin_type', 'title',
       'color', 'estimated_population', 'phase1_population', 'phase1_percent',
       'phase1_color', 'phase2_population', 'phase2_percent', 'phase2_color',
       'phase3_population', 'phase3_percent', 'phase3_color',
       'phase4_population', 'phase4_percent', 'phase4_color',
       'phase5_population', 'phase5_percent', 'phase5_color', 'geometry'],
      dtype='object')

In [104]:
# join all_analysis and joined_gdf on id and aar_id
joined = all_analysis.concat(joined_gdf, on="id", how="inner", lsuffix="_analysis", rsuffix="_joined")


AttributeError: 'DataFrame' object has no attribute 'concat'

In [128]:
# set index of joined_gdf to be aar_id
joined_gdf.set_index("anl_id", inplace=True)
# all_analysis.set_index("id", inplace=True)

In [129]:
joined_gdf.head(1)

,confidence_level,ipc_period,overall_phase,population_min,population_percentage_min,justification,prolonged_crisis,population_phase_worse,population_percentage_phase_worse,admin_type,...,phase3_population,phase3_percent,phase3_color,phase4_population,phase4_percent,phase4_color,phase5_population,phase5_percent,phase5_color,geometry
anl_id,,,,,,,,,,,,,,,,,,,,,
25159242,3,C,3,60410,0.35,Abim district is classified in IPC phase 3 wi...,False,None,None,None,...,51780.0,0.3,#E67800,8630.0,0.05,#C80000,0.0,0.0,#640000,"POLYGON ((33.61183 3.13407, 33.61697 3.13486, ..."


In [110]:
all_analysis.head(1)

,title,link,country,year,condition,created,modified
id,,,,,,,
12135258,Burundi - Acute Food Insecurity March 2017 (20...,http://www.ipcinfo.org/ipc-country-analysis/de...,BI,2017,A,2017-11-20,2022-02-15


In [130]:
# join joined_gdf and all_analysis on aar_id
joined = joined_gdf.join(all_analysis, on="aar_id", how="inner", lsuffix="_joined", rsuffix="_analysis")

KeyError: 'aar_id'

In [113]:
joined.head()

,confidence_level,ipc_period,overall_phase,population_min,population_percentage_min,justification,prolonged_crisis,population_phase_worse,population_percentage_phase_worse,anl_id,...,phase5_percent,phase5_color,geometry,title_analysis,link,country,year,condition,created,modified
aar_id,,,,,,,,,,,,,,,,,,,,,


In [118]:
# print aar_id column of joined_gdf
joined_gdf.index

Index(['25159255', '25159261', '25159253', '25159245', '25159257', '25159251',
       '25159259', '25159249', '25159247', '26468820',
       ...
       '12942124', '24738244', '17950211', '17950203', '17950201', '17950199',
       '17950205', '17950207', '17950213', '17950209'],
      dtype='object', name='aar_id', length=7629)

In [131]:
# join joined_gdf and all_analysis on index
joined = joined_gdf.join(all_analysis, how="inner", lsuffix="_joined", rsuffix="_analysis")

In [132]:
joined

,confidence_level,ipc_period,overall_phase,population_min,population_percentage_min,justification,prolonged_crisis,population_phase_worse,population_percentage_phase_worse,admin_type,...,phase5_percent,phase5_color,geometry,title_analysis,link,country,year,condition,created,modified
12135258,2,C,3,271918,0.3000,La zone est classée en Phase 3 de crise alimen...,False,271918,0.3000,None,...,0.0,#640000,"MULTIPOLYGON (((29.96857 -3.98761, 29.98180 -3...",Burundi - Acute Food Insecurity March 2017 (20...,http://www.ipcinfo.org/ipc-country-analysis/de...,BI,2017,A,2017-11-20,2022-02-15
12135258,2,C,3,709163,0.4000,"En 2017A, il y a eu chute de production de 30%...",False,709163,0.4000,None,...,0.0,#640000,"MULTIPOLYGON (((30.27603 -2.97597, 30.27784 -2...",Burundi - Acute Food Insecurity March 2017 (20...,http://www.ipcinfo.org/ipc-country-analysis/de...,BI,2017,A,2017-11-20,2022-02-15
12135258,2,C,3,47623,0.2000,La zone Buragane est classée en phase 2 avec r...,False,47623,0.2000,None,...,0.0,#640000,"MULTIPOLYGON (((29.84228 -4.21076, 29.84333 -4...",Burundi - Acute Food Insecurity March 2017 (20...,http://www.ipcinfo.org/ipc-country-analysis/de...,BI,2017,A,2017-11-20,2022-02-15
12135258,2,C,2,222576,0.1900,Bien que le rCSI soit en dessous du seuil acce...,False,222575,0.1900,None,...,0.0,#640000,"MULTIPOLYGON (((29.36013 -2.95831, 29.36748 -2...",Burundi - Acute Food Insecurity March 2017 (20...,http://www.ipcinfo.org/ipc-country-analysis/de...,BI,2017,A,2017-11-20,2022-02-15
12135258,2,C,4,293037,0.4500,Les engrais ont été disponibles où 86% des mén...,False,293037,0.4500,None,...,0.0,#640000,"MULTIPOLYGON (((30.10073 -2.42500, 30.09402 -2...",Burundi - Acute Food Insecurity March 2017 (20...,http://www.ipcinfo.org/ipc-country-analysis/de...,BI,2017,A,2017-11-20,2022-02-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26468816,3,C,3,179828,0.45,"Pour la zone de Sud HT-07, la consommation ali...",False,None,None,None,...,0.0,#640000,"MULTIPOLYGON (((-73.10045 18.35305, -73.09679 ...",Haiti - Acute Food Insecurity September 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,HT,2022,A,2022-10-13,2022-10-14
26468816,3,C,4,124423,0.55,"Les ménages pauvres de la zone HT08, qui ont é...",False,None,None,None,...,0.0,#640000,"MULTIPOLYGON (((-73.70372 18.10930, -73.69964 ...",Haiti - Acute Food Insecurity September 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,HT,2022,A,2022-10-13,2022-10-14
26468816,3,C,3,12622,0.4,"La situation socio politique, l’accentuation ...",False,None,None,None,...,0.0,#640000,"MULTIPOLYGON (((-72.25530 18.55421, -72.25575 ...",Haiti - Acute Food Insecurity September 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,HT,2022,A,2022-10-13,2022-10-14
26468816,3,C,3,19074,0.45,"En tenant compte des facteurs contributifs, si...",False,None,None,None,...,0.0,#640000,"POLYGON ((-74.13324 18.66061, -74.13014 18.658...",Haiti - Acute Food Insecurity September 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,HT,2022,A,2022-10-13,2022-10-14


In [127]:
# print all_analysis row where index matches 17950201
all_analysis.loc["25159242"]

title        Uganda - Acute Food Insecurity March 2022
link                                              None
country                                             UG
year                                              2022
condition                                            A
created                                     2022-05-12
modified                                    2022-06-01
Name: 25159242, dtype: object

In [140]:
# write joined to csv
joined_gdf.to_csv("../assets/ipc-data/all_ipc_joined.csv")

# write joined to geojson
joined_gdf.to_file("../assets/ipc-data/all_ipc_joined.geojson", driver="GeoJSON")

In [134]:
# convert joined to geodataframe
joined_gdf = gpd.GeoDataFrame(joined)

In [137]:
# set geopandas geometry column to geometry in joined_gdf
joined_gdf.set_geometry("geometry", inplace=True)

In [144]:
joined_gdf.head(125).explore()

In [147]:
joined_gdf.to_crs(epsg=4326)


,confidence_level,ipc_period,overall_phase,population_min,population_percentage_min,justification,prolonged_crisis,population_phase_worse,population_percentage_phase_worse,admin_type,...,phase5_percent,phase5_color,geometry,title_analysis,link,country,year,condition,created,modified
12135258,2,C,3,271918,0.3000,La zone est classée en Phase 3 de crise alimen...,False,271918,0.3000,None,...,0.0,#640000,"MULTIPOLYGON (((29.96857 -3.98761, 29.98180 -3...",Burundi - Acute Food Insecurity March 2017 (20...,http://www.ipcinfo.org/ipc-country-analysis/de...,BI,2017,A,2017-11-20,2022-02-15
12135258,2,C,3,709163,0.4000,"En 2017A, il y a eu chute de production de 30%...",False,709163,0.4000,None,...,0.0,#640000,"MULTIPOLYGON (((30.27603 -2.97597, 30.27784 -2...",Burundi - Acute Food Insecurity March 2017 (20...,http://www.ipcinfo.org/ipc-country-analysis/de...,BI,2017,A,2017-11-20,2022-02-15
12135258,2,C,3,47623,0.2000,La zone Buragane est classée en phase 2 avec r...,False,47623,0.2000,None,...,0.0,#640000,"MULTIPOLYGON (((29.84228 -4.21076, 29.84333 -4...",Burundi - Acute Food Insecurity March 2017 (20...,http://www.ipcinfo.org/ipc-country-analysis/de...,BI,2017,A,2017-11-20,2022-02-15
12135258,2,C,2,222576,0.1900,Bien que le rCSI soit en dessous du seuil acce...,False,222575,0.1900,None,...,0.0,#640000,"MULTIPOLYGON (((29.36013 -2.95831, 29.36748 -2...",Burundi - Acute Food Insecurity March 2017 (20...,http://www.ipcinfo.org/ipc-country-analysis/de...,BI,2017,A,2017-11-20,2022-02-15
12135258,2,C,4,293037,0.4500,Les engrais ont été disponibles où 86% des mén...,False,293037,0.4500,None,...,0.0,#640000,"MULTIPOLYGON (((30.10073 -2.42500, 30.09402 -2...",Burundi - Acute Food Insecurity March 2017 (20...,http://www.ipcinfo.org/ipc-country-analysis/de...,BI,2017,A,2017-11-20,2022-02-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26468816,3,C,3,179828,0.45,"Pour la zone de Sud HT-07, la consommation ali...",False,None,None,None,...,0.0,#640000,"MULTIPOLYGON (((-73.10045 18.35305, -73.09679 ...",Haiti - Acute Food Insecurity September 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,HT,2022,A,2022-10-13,2022-10-14
26468816,3,C,4,124423,0.55,"Les ménages pauvres de la zone HT08, qui ont é...",False,None,None,None,...,0.0,#640000,"MULTIPOLYGON (((-73.70372 18.10930, -73.69964 ...",Haiti - Acute Food Insecurity September 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,HT,2022,A,2022-10-13,2022-10-14
26468816,3,C,3,12622,0.4,"La situation socio politique, l’accentuation ...",False,None,None,None,...,0.0,#640000,"MULTIPOLYGON (((-72.25530 18.55421, -72.25575 ...",Haiti - Acute Food Insecurity September 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,HT,2022,A,2022-10-13,2022-10-14
26468816,3,C,3,19074,0.45,"En tenant compte des facteurs contributifs, si...",False,None,None,None,...,0.0,#640000,"POLYGON ((-74.13324 18.66061, -74.13014 18.658...",Haiti - Acute Food Insecurity September 2022,https://www.ipcinfo.org/ipc-country-analysis/d...,HT,2022,A,2022-10-13,2022-10-14


In [ ]:
joined_gj = joined_gdf.to_crs(epsg=4326).to_json()

In [148]:
# write joined to csv
joined_gdf.to_csv("../assets/ipc-data/all_ipc_joined-v2.csv")
# write joined to geojson
joined_gdf.to_file("../assets/ipc-data/all_ipc_joined-v2.geojson", driver="GeoJSON")